# 1. 데이터 크롤링

In [10]:
import pandas as pd
from datetime import datetime

from data_crawler import NaverNewsCrawler

/root/.pyenv/versions/3.7.12/envs/demo/lib/python3.7/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [7]:
cralwer = NaverNewsCrawler()
start_date = '2022-07-15'
end_date = datetime.now().strftime('%Y-%m-%d')
cralwer.get_news_date_range(start_date, end_date)

In [ ]:
datelist.remove(end_date) # 오늘자 데이터 삭제 (아직 데이터가 다 들어오지 않음)

# 2. 정제할 언론사 수 확인

In [11]:
import glob

from config import *

In [12]:
glob.glob(CRAWLING_DATA_DIR + '*.csv')

['/root/toyproject/DataCollection/crawling_data/2022-07-25.csv',
 '/root/toyproject/DataCollection/crawling_data/2022-07-26.csv',
 '/root/toyproject/DataCollection/crawling_data/2022-07-27.csv',
 '/root/toyproject/DataCollection/crawling_data/2022-07-28.csv',
 '/root/toyproject/DataCollection/crawling_data/2022-07-29.csv',
 '/root/toyproject/DataCollection/crawling_data/2022-07-30.csv',
 '/root/toyproject/DataCollection/crawling_data/2022-07-31.csv',
 '/root/toyproject/DataCollection/crawling_data/2022-08-01.csv']

In [13]:
df = pd.DataFrame()
for fpath in glob.glob(CRAWLING_DATA_DIR+'*.csv'):
    new_df = pd.read_csv(fpath, header=0, encoding='utf-8')
    date = fpath.split('/')[-1][:-4]
    new_df['date'] = date
    df = pd.concat([df, new_df])
df = df.reset_index(drop=True)
df.shape

(400, 12)

# 3. 데이터 정제

In [16]:
import re
import random

In [17]:
def text_refine(text):
    
    # 엑스포츠 뉴스
    text = re.sub("\(.+기자\)", "", text)
    
    # 인벤
    text = re.sub("▲ 출처: 네이버.+기사제보 및 문의", '', text)
    text = re.sub("e스포츠인벤 Copyright.+기사제보 및 문의", '', text)
    
    # 포모스
    text = re.sub("포모스와 함께 즐기는 e스포츠,.+무단 +전재 +및 +재배포 +금지 +포모스", "", text)
    
    # 스포츠 서울
    text = re.sub("(사진)*제공.+ 기자\]", '', text)
    
    # 국민일보
    text = re.sub("\[네이버 메인에서 채널 구독하기\].+재배포금지", "", text)
    text = re.sub("\[국민일보 채널 구독하기\].+재배포금지", "", text)
    text = re.sub('LCK 제공', "", text)
    
    # 공통
    text = re.sub("^사진( 출처)*=[가-힣a-zA-Z0-9 ]+(\,|\.)", "", text)
    text = re.sub("\xa0", " ", text)
    text = re.sub('\[[^\]]+\]', " ", text)
    photos = re.findall("사진=.+$", text)
    if photos:
        if len(photos[0]) < 20:
            text = re.sub("사진=.+$", "", text)
    text = re.sub(r"((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*", "", text)

    # 
    text = re.sub("[^ㄱ-ㅎ가-힣0-9a-zA-Z\.\?\!\[\]\-\/\:\%\(\) ]+", "", text)
    text = re.sub("\(\)", '', text)
    text = re.sub(' {2,}', ' ', text).strip()
    text = re.sub('[가-힣a-zA-Z\,\/\ ]+ 기자$', '', text).strip()
    text = text.strip("/").strip()
    
    return text

In [25]:
# new_df = df[df["officeName"].apply(lambda x: ('포모스' not in x) & ('인벤' not in x) & \
#                                               ('엑스포츠' not in x) & ('서울' not in x) & \
#                                               ('국민일보' not in x))].reset_index(drop=True)
new_df = df[df["officeName"].apply(lambda x: ('한국경제' in x))].reset_index(drop=True)
idx = random.randint(0, len(new_df['content'])-1)

# idx = 40

print(idx)
print(new_df.iloc[idx]['officeName'])
print('-'*50)
text = new_df['content'][idx]
print(text)
print('-'*50)

text = text_refine(text)

print(text)

0
한국경제
--------------------------------------------------
[이주현의 로그인 e스포츠] 는 게임을 넘어 스포츠, 그리고 문화콘텐츠로 성장하고 있는 e스포츠에 대한 이야기를 다룹니다. 인상 깊었던 경기들은 물론, 궁금했던 뒷이야기 나아가 산업으로서 e스포츠의 미래에 대해 분석합니다. 젠지 선수단(제공=LCK) 2022 LCK(리그오브레전드 챔피언스 코리아) 서머 스플릿 최강자가 오늘 가려진다. 정규리그 1위 자리를 놓고 치열하게 경쟁 중인 T1과 젠지e스포츠가 30일 맞붙는다. 두 팀은 현재까지 각각 12승 1패로 3,4위권 팀과 큰 격차를 벌리며 압도적인 실력을 뽐내고 있다. 양 팀이 남은 경기에서 모두 이긴다면 오늘 경기의 승자가 사실상 1위가 된다. 젠지는 명실상부한 강팀이다. 이번 시즌 T1과 담원 기아를 제외한 모든 팀을 상대로 2대0 완승을 거뒀다. 담원을 상대로도 1라운드 2대1로 승리했다. 현재까지 세트 기준 25승 3패로 강력함을 뽐내고 있다. 팀 지표 측면에서도 평균 경기 시간이 30분 19초로 가장 짧고 평균 킬은 14.4로 가장 많다. 말 그대로 폭발적인 경기력을 보여주고 있다. 이를 보여주듯 젠지 선수들 역시 각종 지표에서 최상위권에 올라 있다. T1 선수단(제공=LCK) 그런 젠지에 서머 시즌 유일한 패배를 안긴 팀이 바로 T1이다. 젠지 입장에서 T1은 트라우마에 가깝다. 2021년 LCK 서머 스플릿부터 현재까지 상대 전적이 7전 전패다. 이번 시즌뿐 아니라 올해 내내 중요한 순간에 T1을 넘지 못했다. 스프링 스플릿에서 젠지는 15승 3패로 일반적이라면 리그 1위를 다툴 만한 퍼포먼스를 보였다. 하지만 18승 0패로 전승을 달성한 T1에 밀려 2위에 그쳤다. 스프링 결승전에서도 3대1로 패하며 아쉬움을 삼켰다. 젠지가 오늘 T1전의 악몽에서 벗어나기 위해선 도란(최현준)의 역할이 중요하다. ‘T1의 상수’라고 불리는 제우스(최우제)를 막아내야 하기 때문이다. 젠지의 도란(최현준, 왼쪽)과

In [20]:
# refined_text 칼럼 생성
df['refined_text'] = df['content'].apply(lambda x: text_refine(x))

# 4. 데이터 토큰화
1) mecab
    
    - 띄어쓰기 들어간 복합명사도 사용자 정의 사전에 추가 가능
    
2) <s>huggingface tokenizer</s>

In [23]:
# mecab
from konlpy.tag import Mecab
m = Mecab()
pos_text = m.pos(df.iloc[3]['refined_text'])
print(pos_text)

[('T1', 'NNG'), ('오너', 'NNG'), ('문현준', 'NNP'), ('이', 'JKS'), ('자신', 'NNG'), ('의', 'JKG'), ('시그', 'NNP'), ('니', 'VCP+EC'), ('처', 'NNG'), ('픽', 'NNG'), ('인', 'VCP+ETM'), ('리신', 'NNG'), ('에', 'JKB'), ('대해서', 'VV+EC+VCP'), ('고', 'EC'), ('평가', 'NNG'), ('했', 'XSV+EP'), ('다', 'EF'), ('.', 'SF'), ('T1', 'NNG'), ('은', 'JX'), ('1', 'SN'), ('일', 'NNBC'), ('서울', 'NNP'), ('종로구', 'NNP'), ('그랑', 'NNP'), ('서울', 'NNP'), ('LCK', 'SL'), ('아레나', 'NNG'), ('에서', 'JKB'), ('진행', 'NNG'), ('된', 'XSV+ETM'), ('2022', 'SN'), ('리그', 'NNG'), ('오브', 'NNP'), ('레전드', 'NNP'), ('챔피', 'NNP'), ('언스', 'NNP'), ('코리아', 'NNP'), ('서머', 'NNP'), ('1', 'SN'), ('라운드', 'NNG'), ('에서', 'JKB'), ('디알엑스', 'NNG'), ('를', 'JKO'), ('2', 'SN'), ('대', 'NNBC'), ('0', 'SN'), ('으로', 'JKB'), ('제압', 'NNG'), ('했', 'XSV+EP'), ('다', 'EF'), ('.', 'SF'), ('문현준', 'NNP'), ('은', 'JX'), ('1', 'SN'), ('세트', 'NNG'), ('에서', 'JKB'), ('는', 'JX'), ('리신', 'NNG'), ('2', 'SN'), ('세트', 'NNG'), ('에서', 'JKB'), ('는', 'JX'), ('비에고', 'NNG'), ('로', 'JKB'), ('맹활약', 'NNG'), 

In [30]:
word_list = []
tags = ["NNG", "NNP"]
for word, tag in pos_text:
    cond1= tag in tags
    cond2 = (tag == 'SL') & (len(word) > 1)
    if cond1 | cond2:
        word_list.append(word)

In [31]:
print(word_list)

['T1', '오너', '문현준', '자신', '시그', '처', '픽', '리신', '평가', 'T1', '서울', '종로구', '그랑', '서울', 'LCK', '아레나', '진행', '리그', '오브', '레전드', '챔피', '언스', '코리아', '서머', '라운드', '디알엑스', '제압', '문현준', '세트', '리신', '세트', '비에고', '맹활약', '팀', '승리', '경기', '후', '인터뷰', '문현준', '개인', '리신', '만능', '생각', '오공', '비에고', '티어', '설명', '연승', '부담감', '연승', '아쉬움', '연승', '아쉬움', '다음', '문현준', '일문일답', '승리', '소감', '경기', '연승', '순항', '디알엑스', '상대', '완승', '다행', '기분', '단독', 'POG', '본인', '경기력', '평가', '정도', '긴장', '스킬', '경우', '아쉬움', '긴장', '이유', '특별', '이유', '게임', '진행', '경기', '패배', '원인', '상대', '준비', '픽', '예상', '광동', '준비', '게임', '제우스', '최우제', '호흡', '생각', '최우제', '아마추어', '생활', '지금', '호흡', '생각', '최우제', '연승', '마음', '연승', '부담감', '연승', '아쉬움', '리신', '평가', '공격력', '버프', 'AD', '정글', '상대', '개인', '리신', '만능', '생각', '오공', '비에고', '티어', '리신', '플레이', '집중', '생각', '상대', '딜러', '리신', '압박', '불편', '위치', '중요', '궁', '배달', '아군', '궁', '사용', '가능', '데뷔', '기억', '순간', '월드', '챔피언십', '롤', '드', '컵', '강', '순간', '기억', '마지막', '데뷔', '기분', '팬', '감사', '날']
